## DDPM Generate an image using Diffusers 

- Reference [diffusers_intro](https://github.com/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb), [training_example](https://github.com/huggingface/notebooks/blob/main/diffusers/training_example.ipynb)
- Requirements: diffusers, pytorch 

### Installation
- Install `!pip install diffusers==0.11.1`
- Install `!pip install accelerate`  
  if you see the following message    
  ```
  Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
    - Install `pip install diffusers[training]==0.11.1
  ```

In [ ]:
from IPython.display import Image as DisplayImage

from diffusers import DDPMPipeline

import torch
import time

In [ ]:
dataset_path = '/group-volume/sr_edu/AI-Application-Specialist-Vision-Dataset/'

DisplayImage(filename=dataset_path  + 'hf-assets/ddpm_paper.png', width=600)

In [ ]:
# in order to download models from huggingface, it is necessary to set the following proxy and ssl 
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# suwon
import os
os.environ['REQUESTS_CA_BUNDLE'] = '/etc/ssl/certs/ca-certificates.crt'
os.environ['HTTP_PROXY'] ='http://75.17.107.42:8080'
os.environ['HTTPS_PROXY'] ='http://75.17.107.42:8080'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
hf_model_dir = dataset_path + "hf-models/"

In [ ]:
ddpm_pipe = DDPMPipeline.from_pretrained(hf_model_dir+"google/ddpm-cat-256")
#ddpm_pipe = DDPMPipeline.from_pretrained(hf_model_dir+"google/ddpm-celebahq-256")
# directly to download from huggingface 
#ddpm_pipe = DDPMPipeline.from_pretrained("google/ddpm-celebahq-256")
ddpm_pipe.to(device)
ddpm_pipe

In [ ]:
images = ddpm_pipe().images

In [ ]:
images[0]

In [ ]:
len(images)

In [ ]:
from diffusers import UNet2DModel

repo_id = hf_model_dir + "google/ddpm-church-256"
model = UNet2DModel.from_pretrained(repo_id)

In [ ]:
model

In [ ]:
model.config

In [ ]:
# random image at time=T
torch.manual_seed(0)

noisy_sample = torch.randn(
    1, model.config.in_channels, model.config.sample_size, model.config.sample_size
)
noisy_sample.shape

In [ ]:
with torch.no_grad():
    noisy_residual = model(sample=noisy_sample, timestep=2).sample

In [ ]:
from diffusers import DDPMScheduler

scheduler = DDPMScheduler.from_config(repo_id)

In [ ]:
scheduler.config

In [ ]:
less_noisy_sample = scheduler.step(
    model_output=noisy_residual, timestep=2, sample=noisy_sample
).prev_sample

In [ ]:
noisy_residual.shape, less_noisy_sample.shape

In [ ]:
import PIL.Image
import numpy as np

def display_sample(sample, i):
    image_processed = sample.cpu().permute(0, 2, 3, 1)
    image_processed = (image_processed + 1.0) * 127.5
    image_processed = image_processed.numpy().astype(np.uint8)

    image_pil = PIL.Image.fromarray(image_processed[0])
    display(f"Image at step {i}")
    display(image_pil)

In [ ]:
display_sample(less_noisy_sample,2)

In [ ]:
model.to(device)
noisy_sample = noisy_sample.to(device)

In [ ]:
import tqdm

sample = noisy_sample

for i, t in enumerate(tqdm.tqdm(scheduler.timesteps)):
    # 1. predict noise residual
    with torch.no_grad():
        residual = model(sample, t).sample

    # 2. compute less noisy image and set x_t -> x_t-1
    sample = scheduler.step(residual, t, sample).prev_sample

    # 3. optionally look at image
    if (i + 1) % 50 == 0:
        display_sample(sample, i + 1)

### DDIM Scheduler

In [ ]:
from diffusers import DDIMScheduler

scheduler = DDIMScheduler.from_config(repo_id)

In [ ]:
scheduler.set_timesteps(num_inference_steps=50)

In [ ]:
import tqdm

sample = noisy_sample

for i, t in enumerate(tqdm.tqdm(scheduler.timesteps)):
    # 1. predict noise residual
    with torch.no_grad():
        residual = model(sample, t).sample

    # 2. compute previous image and set x_t -> x_t-1
    sample = scheduler.step(residual, t, sample).prev_sample

    # 3. optionally look at image
    if (i + 1) % 10 == 0:
        display_sample(sample, i + 1)

### cifar10-ddpm

In [ ]:
# !pip install diffusers
from diffusers import DDPMPipeline, DDIMPipeline, PNDMPipeline

In [ ]:
model_id = hf_model_dir + "google/ddpm-cifar10-32"

In [ ]:
# load model and scheduler
ddpm = DDPMPipeline.from_pretrained(model_id)  # you can replace DDPMPipeline with DDIMPipeline or PNDMPipeline for faster inference
ddpm = ddpm.to(device)

In [ ]:
# run pipeline in inference (sample random noise and denoise)
t0 = time.time()
image = ddpm().images[0]
print(time.time()-t0)

image

In [ ]:
ddim = DDIMPipeline.from_pretrained(model_id)  # you can replace DDPMPipeline with DDIMPipeline or PNDMPipeline for faster inference
ddim = ddim.to(device)

In [ ]:
# run pipeline in inference (sample random noise and denoise)
t0 = time.time()
image = ddim().images[0]
print(time.time()-t0)

# to save image
# image.save("ddim_generated_image.png")

In [ ]:
image